## BCI Competition 4 Dataset 2a Results

All results are test accuracy from 5-fold cross validation (in percent, mean over all 9 subjects). I show both blockwise cross validation, i.e., the test fold is one "timeblock" with all trials of a certain timeperiod, as well as shuffled cross validation, where the test fold can consist of any randomly selected trials from the data (but still has no trials in common with the train fold). I also show train-test split as in the competition, that means about 40% less training trials. We predict trialwise, so we do not do any continous samplewise prediction (I assume they did the same in their paper).


The train folds have sizes $\frac{4}{5} \cdot 576 \approx 461$, test folds $\frac{1}{5} \cdot 576 \approx 115$.
For train test split, train and test both have 288 trials.

I report for three different forward/causal highpass filters, explanation below.

|Method|Highpass (Hz)|Sampling Rate (Hz)|Blockwise (%)|Shuffled (%)|Train-Test-Split|Windowed Training**|***Windowed Model|
|-|-|-|
|CSP+CNN =>SFM (theirs)||250||69.27|||
| FBCSP (theirs, 0.5-2.5s)||250|||67.01||
| FBCSP (ours, 0.5-2.5s)||250|72.59|73.59|62.77||
| FBCSP (ours, 0.5-2.5s), w/ fsel*||250|||64.20||
| FBCSP (ours, 0.5-4s)||250|76.64|77.72|65.47||
| FBCSP (ours, 0.5-4s), w/ fsel*||250|||67.17||
|Raw Net (0-4s)|0.5 |150|76.19|77.78|69.25||
|Raw Net (0-4s)|2.0 |150|71.78|72.68|66.17||
|Raw Net (0-4s)|4.0 |150|69.89|71.18|64.54|63.43|64.66|
|Raw Net No Drop**** (0-4s)|0.5 |150|||74.07||
|Raw Net No Drop**** (0-4s)|2.0 |150|||73.46||
|Raw Net No Drop**** (0-4s)|4.0 |150|||73.38|||
|Raw Net (0-4s)|0.5 |250|76.75|77.66|69.33||
|Raw Net (0-4s)|2.0 |250|71.07|72.45|65.59||
|Raw Net (0-4s)|4.0 |250|70.28|70.77|64.81|62.85|66.09|
|Raw Net No Drop**** (0-4s)|0.5 |250|||73.69||
|Raw Net No Drop**** (0-4s)|2.0 |250|||74.34||
|Raw Net No Drop**** (0-4s)|4.0 |250|||70.06|||
|FB Net (0.5-4s)|0.5 |150|74.04|75.79|59.57||
|FB Net (0.5-4s)|2.0 |150|73.90|75.48|59.68||
|FB Net (0.5-4s)|4.0 |150|72.30|74.36|57.79|66.44|
|FB Net (0.5-4s)|0.5 |250|72.41|74.81|61.00||
|FB Net (0.5-4s)|2.0 |250|71.70|73.07|60.34||
|FB Net (0.5-4s)|4.0 |250|71.35|72.34|57.99|67.21|

* *w/ fsel means with feature selection
* ** Windowed Training means I cut out windows of 2s from the 4s trials, shifting them through the trials with 100ms stride. It also was evaluated with the train test splits (using the sum over all windows predictions per trial)
* *** Windowed model means the final fully connected layer is replaced by a convolutional layer + averaging over all convolution outputs. This translates to making a prediction over a time region, in our case of roughly 2 seconds. 
* **** Raw Net No Drop means no dropout between the first and second convolutional layer (time convolution and spatial convolution, they have no/identity activation between them). 

###Notes:

* Results with *~* are estimations based on finished datasets. FB Net with 150 Hz shuffled should be finished by tuesday, with 250 Hz finished until end of week.
* They use 500-2500ms, we use 0-4000ms. There was no constraint from the original dataset description. In the competition you originally had to predict samplewise, not trialwise, e.g. by shifting a window over all samples. Only constraint was to only use data from the past up to the current sample to predict the current sample. I could rerun with their timewindow if wanted. I originally ran with >0.5 Hz and their timewindow and 150 Hz and had 74.08% for raw net and 73.86% for fb net.
* FBCSP our implementation also uses same 9 filterbands 4-8,8-12,...,36-40
* FB Net with higher highpass, also has less frequency bands, removing band from 0.5-1Hz and from 1-3Hz respectively.
* EOG artifacts need to be filtered out, maybe we use them? Quote from dataset description for competition:
> Since three EOG channels are provided, it is required to remove EOG artifacts before the subsequent data processing using artifact removal techniques such as highpass filtering or linear regression.  In order to enable the application of other correction methods, we have opted for a maximum transparency approach and provided the EOG channels; at the same time we request that artifacts do not influence the classication result.

     * At the moment I do not reject any trials or channels. This is the easiest setup as I do not have to change anything for running it on the unclean test set. I also assume they do not do any cleaning and that they believe their frequency bands (between 4 and 40 Hz) are relatively artifact free. So probably only our results above 4 Hz are comparable.
* We do not know if they shuffled or didn't shuffle for their cross validation.
* I do the highpass before even for the fb net, where it does not make much sense (since there is a bandpass after). This is just a legacy from the master thesis idea that every net should have as identical an input as possible. I can rerun fb net without highpass before, although that should not make a big difference (I think?), only that maybe minimally less relevant samples are shifted out of the time window by only doing forward filtering once.
* Nets are identical to master thesis except for weight initialization

## Details

All subjects for fb net/raw net 250 Hz train/test compared to FBCSP.

|Subject|FBCSP|Raw|FB|
|-|-|-|-|-|
| S1 |79.16|76.39|62.50|
| S2 |52.08|42.71|33.33|
| S2 |83.33|85.42|75.00|
| S4 |62.15|60.42|60.07|
| S5 |54.51|35.76|48.26|
| S6 |39.24|50.35|42.36|
| S7 |83.33|85.42|65.62|
| S8 |82.64|71.88|69.10|
| S9 |66.67|75.00|65.62|
|Avg |67.01|64.81|57.99|